In this notebook we have trained the BERT model. As we are using a macbook pro 2019 with an Intel video card, we have not the possibility to use GPU capability by laveriging e.g., cuda driver on an Nvidia video card. For this reason the below code has actually been processed over Google colab workspace. 

This notebook has to be considered as follow up of the Toxic Comment Analysis one where we have processed all other models. 

### 6. BERT

In [2]:
! pip install pytorch_pretrained_bert pytorch-nlp -q

     |████████████████████████████████| 123 kB 5.3 MB/s 
     |████████████████████████████████| 90 kB 8.1 MB/s 
     |████████████████████████████████| 131 kB 41.9 MB/s 
     |████████████████████████████████| 8.0 MB 43.4 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 138 kB 44.1 MB/s 
     |████████████████████████████████| 127 kB 27.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


#### a. Import libraries

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from bs4 import BeautifulSoup
import re

from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
import torch
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, DataLoader
from torch.nn import BCEWithLogitsLoss, Sigmoid

from tqdm.notebook import tqdm, trange

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

#
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

#### b. Read dataset

In [5]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")
sample_submission = pd.read_csv("/content/sample_submission.csv")

print(df_train.shape, df_test.shape, sample_submission.shape)
df_train.head()

(159571, 8) (153164, 2) (153164, 7)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


#### c. Pre-trained dataset

In [6]:
def strip(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = re.sub('\[[^]]*\]', '', soup.get_text())
    pattern=r"[^a-zA-z0-9\s,']"
    text=re.sub(pattern,'',text)
    return text

df_train["comment_text"] = df_train["comment_text"].apply(strip)
df_test["comment_text"] = df_test["comment_text"].apply(strip)

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://en.wikipedia.org/wiki/Wikipedia_talk:No_original_research/archive15#YouTube_art_as_primary_source" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://finance.yahoo.com/news/7-fascinating-nuggets-another-bewildering-150348488.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://en.wikipedia.org/wiki/Wikipedia:ELYES" looks like a URL. Beautiful Soup is not an HTTP client

In [7]:
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww He matches this background colour I'm se...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war It'...",0,0,0,0,0,0
3,0001b41b1c6bb37e,\nMore\nI can't make any real suggestions on i...,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero Any chance you remember ...",0,0,0,0,0,0


Preparaing data for BERT input

In [8]:
train_sentences = df_train["comment_text"]
test_sentences = df_test["comment_text"]
train_sentences = ["[CLS] "+ i + " [SEP]"for i in train_sentences]
test_sentences = ["[CLS] "+ i + " [SEP]"for i in test_sentences]
train_sentences[0], test_sentences[0]

("[CLS] Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC And please don't remove the template from the talk page since I'm retired now892053827 [SEP]",
 "[CLS] Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckasi should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me Ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother toofuckin white boys get things right next time, [SEP]")

To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary. The tokenization must be performed by the tokenizer included with BERT. We’ll be using the “uncased” version here.

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

train_tokenizer_texts = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(train_sentences)))

test_tokenizer_texts = list(map(lambda t: tokenizer.tokenize(t)[:510], tqdm(test_sentences)))

np.array(train_tokenizer_texts[0]), np.array(test_tokenizer_texts[0])

100%|██████████| 231508/231508 [00:00<00:00, 2541358.42B/s]


  0%|          | 0/159571 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

(array(['[CLS]', 'explanation', 'why', 'the', 'edit', '##s', 'made',
        'under', 'my', 'user', '##name', 'hardcore', 'metallic', '##a',
        'fan', 'were', 'reverted', 'they', 'weren', "'", 't', 'van',
        '##dal', '##isms', ',', 'just', 'closure', 'on', 'some', 'gas',
        'after', 'i', 'voted', 'at', 'new', 'york', 'dolls', 'fa', '##c',
        'and', 'please', 'don', "'", 't', 'remove', 'the', 'template',
        'from', 'the', 'talk', 'page', 'since', 'i', "'", 'm', 'retired',
        'now', '##8', '##9', '##20', '##53', '##8', '##27', '[SEP]'],
       dtype='<U11'),
 array(['[CLS]', 'yo', 'bitch', 'ja', 'rule', 'is', 'more', 'su', '##cc',
        '##es', '##ful', 'then', 'you', "'", 'll', 'ever', 'be', 'what',
        '##s', 'up', 'with', 'you', 'and', 'hating', 'you', 'sad', 'mo',
        '##fu', '##ck', '##asi', 'should', 'bitch', 'slap', 'ur', 'pet',
        '##hed', '##ic', 'white', 'faces', 'and', 'get', 'you', 'to',
        'kiss', 'my', 'ass', 'you', 'guys', 

In [10]:
labels = df_train[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].to_numpy()

The sentences in our dataset obviously have varying lengths. BERT has two constraints:

All sentences must be padded or truncated to a single, fixed length.
The maximum sentence length is 512 tokens. Padding is done with a special [PAD] token, which is at index 0 in the BERT vocabulary.

In [11]:
#Padding/truncating the train and test sentences to a size of 128 tokens per sentence

MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(train_tokenizer_texts)]
input_ids = pad_sequences(sequences = input_ids, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')

test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(test_tokenizer_texts)]
test_input_ids = pad_sequences(sequences = test_input_ids, maxlen = MAX_LEN, dtype = 'long', padding='post', truncating='post')


input_ids[0], test_input_ids[0]

  0%|          | 0/159571 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

(array([  101,  7526,  2339,  1996, 10086,  2015,  2081,  2104,  2026,
         5310, 18442, 13076, 12392,  2050,  5470,  2020, 16407,  2027,
         4694,  1005,  1056,  3158,  9305, 22556,  1010,  2074,  8503,
         2006,  2070,  3806,  2044,  1045,  5444,  2012,  2047,  2259,
        14421,  6904,  2278,  1998,  3531,  2123,  1005,  1056,  6366,
         1996, 23561,  2013,  1996,  2831,  3931,  2144,  1045,  1005,
         1049,  3394,  2085,  2620,  2683, 11387, 22275,  2620, 22907,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [12]:
#Creating an attention mask - For actual tokens its set to 1, for padding tokens its set to 0
def create_attention_masks(input_ids):
    attention_masks = []
    for seq in tqdm(input_ids):
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return np.array(attention_masks)

attention_masks = create_attention_masks(input_ids)
test_attention_masks = create_attention_masks(test_input_ids)
attention_masks[0], test_attention_masks[0]

  0%|          | 0/159571 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.

In [13]:
# Use train_test_split to split our data and attention masks into train and validation sets for training
X_train, X_val, y_train, y_val = train_test_split(input_ids, labels, random_state = 123, test_size = 0.20)
attention_masks_train, attention_masks_val = train_test_split(attention_masks, random_state = 123, test_size = 0.20)

In [14]:
# Convert all inputs and labels into torch tensors, the required datatype 
X_train = torch.tensor(X_train)
X_val = torch.tensor(X_val)
y_train = torch.tensor(y_train) 
y_val = torch.tensor(y_val)
attention_masks_train = torch.tensor(attention_masks_train)
attention_masks_val = torch.tensor(attention_masks_val)

test_input_ids = torch.tensor(test_input_ids)
test_attention_masks = torch.tensor(test_attention_masks)

In [15]:
y_val.shape

torch.Size([31915, 6])

We’ll also create an iterator for our dataset using the torch DataLoader class. This helps save on memory during training because, unlike a for loop, with an iterator the entire dataset does not need to be loaded into memory. In Training we need to randomize the data for generalizing better so we use random sampler. We dont need that while predicting in validation or test set so we use sequential sampler.

In [16]:
BATCH_SIZE = 32
#Dataset wrapping tensors.
train_data = TensorDataset(X_train, attention_masks_train, y_train)
val_data = TensorDataset(X_val, attention_masks_val, y_val)
test_data = TensorDataset(test_input_ids, test_attention_masks)
#Samples elements randomly. If without replacement(default), then sample from a shuffled dataset.
train_sampler = RandomSampler(train_data)
val_sampler = SequentialSampler(val_data)
test_sampler = SequentialSampler(test_data)
#represents a Python iterable over a dataset
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = BATCH_SIZE)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size = BATCH_SIZE)

#### d. Loading pre trained BERT and setting fine-tuning parameters

First we load the bert model for sequence classification. We set the output neurons to 6 as we have 6 toxic types to be predicted for as yes or no.

Next, we get weights for various layers and put them into a single list. Once that is done, we separate weight parameters (which needs to be updated) from bias, gamma, and beta parameters (which don't need to be updated). We filter one group without these values and another with them. Hence you see one group with weight_decay_rate 0.01 and another 0.0.

In [17]:
#Inititaing a BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 6)
model.cuda()

100%|██████████| 407873900/407873900 [00:11<00:00, 36420128.19B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [18]:
#Dividing the params into those which needs to be updated and rest
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01
    },
    {
        'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0
    }
]

optimizer = BertAdam(optimizer_grouped_parameters, lr = 2e-5, warmup = .1)

t_total value of -1 results in schedule not being applied


Now that we have our model loaded we need to grab the training hyperparameters from within the stored model. For the purposes of fine-tuning, the authors recommend choosing from the following values:

Batch size: 16, 32 (We chose 32 when creating our DataLoaders).

Learning rate (Adam): 5e-5, 3e-5, 2e-5 (We’ll use 2e-5).

Number of epochs: 2, 3, 4 (We’ll use 2).

In [19]:
#freeing up memory
torch.cuda.empty_cache()
import gc
gc.collect()

0

#### d. Tunning BERT

This is the phase where we fine-tune the BERT model on our dataset for as much epochs as required and validate the model performance on Validation data

We will be using BCEWithLogitLoss function for calculating multi-label loss between predicted and actual values. This is similar to adding sigmoid function at the end of the network and calculating binary-cross entropy loss like in Keras we did for LSTM and CNN

In [20]:
#Empty the GPU memory as it might be memory and CPU intensive while training
torch.cuda.empty_cache()
#Number of times the whole dataset will run through the network and model is fine-tuned
epochs = 2
#Iterate over number of epochs
for _ in trange(epochs, desc = "Epoch"):
    #Switch model to train phase where it will update gradients
    model.train()
    #Initaite train and validation loss, number of rows passed and number of batches passed
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    val_loss = 0
    nb_val_examples, nb_val_steps = 0, 0
    #Iterate over batches within the same epoch
    for batch in tqdm(train_dataloader):
        #Shift the batch to GPU for computation
        batch = tuple(t.to(device) for t in batch)
        #Load the input ids and masks from the batch
        b_input_ids, b_input_mask, b_labels = batch
        #Initiate gradients to 0 as they tend to add up
        optimizer.zero_grad()
        #Forward pass the input data
        logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
        #We will be using the Binary Cross entropy loss with added sigmoid function after that in BCEWithLogitsLoss
        loss_func = BCEWithLogitsLoss()
        #Calculate the loss between multilabel predicted outputs and actuals
        loss = loss_func(logits, b_labels.type_as(logits))
        #Backpropogate the loss and calculate the gradients
        loss.backward()
        #Update the weights with the calculated gradients
        optimizer.step()
        #Add the loss of the batch to the final loss, number of rows and batches
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
    #Print the current training loss 
    print("Train Loss: {}".format(tr_loss/nb_tr_examples))
    #Switch the model to evaluate stage at which the gradients wont be updated
    model.eval()
    #Iterate over the validation data
    for step, batch in enumerate(val_dataloader):
        #Shift the validation data to GPUs for computation
        batch = tuple(t.to(device) for t in batch)
        #We dont want to update the gradients
        with torch.no_grad():
            #Load the input ids and masks from the batch
            b_input_ids, b_input_mask, b_labels = batch
            #Forward pass the input data
            logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
            #We will be using the Binary Cross entropy loss with added sigmoid function after that in BCEWithLogitsLoss
            loss_func = BCEWithLogitsLoss()
            #Calculate the loss between multilabel predicted outputs and actuals
            loss = loss_func(logits, b_labels.type_as(logits))
            #Add the loss of the batch to the final loss, number of rows and batches
            val_loss += loss.item()
            nb_val_examples += b_input_ids.size(0)
            nb_val_steps += 1
    #Print the current validation loss     
    print("Valid Loss: {}".format(val_loss/nb_val_examples))


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3990 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1025.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Train Loss: 0.0015603492090074198
Valid Loss: 0.0013186882466317675


  0%|          | 0/3990 [00:00<?, ?it/s]

Train Loss: 0.0010944415365687103
Valid Loss: 0.00121498617592412


#### e. Predicting and submitting test data

In [ ]:
outputs = []

#Iterate over the test_loader 
for step, batch in enumerate(test_dataloader):
        #Transfer batch to GPUs
        batch = tuple(t.to(device) for t in batch)
        #We dont need to update gradients as we are just predicting
        with torch.no_grad():
            #Bring up the next batch of input_texts and attention_masks 
            b_input_ids, b_input_mask = batch
            #Forward propogate the inputs and get output as logits
            logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
            #Pass the outputs through a sigmoid function to get the multi-label preditions
            s = Sigmoid()
            out = s(logits).to('cpu').numpy()    
            #Add the predictions for this batch to the final list
            outputs.extend(out)
            
#Merge test df and submission table to have all columns in a table
df_test = pd.merge(df_test, sample_submission, on = "id")
#Assign the predictions to the toxic_output columns
df_test[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]] = outputs
#Drop text data as it is not expected in the submission file
df_test.drop(["comment_text"], axis = 1, inplace = True)
#Saving the submission dataframe
df_test.to_csv("/content/sample_submission_BERT.csv", index = False)